In [2]:
import twython
from twython import Twython
import pandas as pd
import json
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

keys = json.load(open("keys.json"))

CONSUMER_KEY = keys['consumer_key']                 # Type your CONSUMER_KEY
CONSUMER_SECRET = keys['consumer_secret']           # Type your CONSUMER_SECRET
OAUTH_TOKEN = keys['access_token']                 # Type your OAUTH_TOKEN
OAUTH_TOKEN_SECRET = keys['access_token_secret']   # Type your OAUTH_TOKEN_SECRET

twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

dataset = pd.read_csv('C:/dataset/2013_pakistan_eq.csv', header=0)

# Retrieving all tweet ids in order to use APIs to fetch all the informations we need on them

tweet_ids = dataset["tweet_id"]

print(type(tweet_ids))

# Removing the apexes from each Tweet ID

for i in range(0, tweet_ids.size):
    tweet_ids[i] = tweet_ids[i].replace("'", "")

print("[DEBUG] Formatting tweets")

# Create a new DataFrame, initially containing just the IDs

# tweet_infos = pd.DataFrame(tweet_ids)

# List that will contain the new informations to add to the DataFrame tweet_infos

tweet_infos = {}
list_infos = []


<class 'pandas.core.series.Series'>
[DEBUG] Formatting tweets


In [3]:
# Test for multiple tweets

for i, id in enumerate(tweet_ids):
    try:
        tweet = twitter.show_status(id=id)
        tweet_infos["ID"] = id
        tweet_infos["Followers"] = tweet['user']['followers_count']
        list_infos.append(tweet_infos)
        print("[DEBUG] Found info for tweet: ", id, ". Added to list")
    except twython.exceptions.TwythonError as e:
        print(e)    # If an exception occurs (APIs return an unexpected HTTP response code) we print it
        tweet_infos["ID"] = id
        tweet_infos["Followers"] = None
        list_infos.append(tweet_infos)
        pass        # Exception is not handled yet

[DEBUG] Found info for tweet:  382813388503412736 . Added to list
[DEBUG] Found info for tweet:  382813391435206659 . Added to list
[DEBUG] Found info for tweet:  382813401845485568 . Added to list
[DEBUG] Found info for tweet:  382813437513834497 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
[DEBUG] Found info for tweet:  382813444011204608 . Added to list
[DEBUG] Found info for tweet:  382813445495586816 . Added to list
[DEBUG] Found info for tweet:  382813466014121984 . Added to list
[DEBUG] Found info for tweet:  382813467528282112 . Added to list
[DEBUG] Found info for tweet:  382813501489946624 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
[DEBUG] Found info for tweet:  382813514881982464 . Added to list
[DEBUG] Found info for tweet:  382813518602313729 . Added to list
[DEBUG] Found info for tweet:  382813530396700672 . Added to list
[DEBUG] Found info for tweet:  382813532305096705 . Added to list
[D

Twitter API returned a 403 (Forbidden), User has been suspended.
Twitter API returned a 403 (Forbidden), User has been suspended.
Twitter API returned a 403 (Forbidden), User has been suspended.
Twitter API returned a 403 (Forbidden), User has been suspended.
[DEBUG] Found info for tweet:  382814942023262208 . Added to list
[DEBUG] Found info for tweet:  382814949988249600 . Added to list
[DEBUG] Found info for tweet:  382814975292481537 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
Twitter API returned a 403 (Forbidden), User has been suspended.
Twitter API returned a 404 (Not Found), No status found with that ID.
[DEBUG] Found info for tweet:  382814998864478208 . Added to list
[DEBUG] Found info for tweet:  382815020603543552 . Added to list
[DEBUG] Found info for tweet:  382815028006514688 . Added to list
[DEBUG] Found info for tweet:  382815045748400128 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
[DE

[DEBUG] Found info for tweet:  382816648341630977 . Added to list
[DEBUG] Found info for tweet:  382816679132020736 . Added to list
[DEBUG] Found info for tweet:  382816685335801856 . Added to list
[DEBUG] Found info for tweet:  382816715907678209 . Added to list
[DEBUG] Found info for tweet:  382816752398528512 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
[DEBUG] Found info for tweet:  382816763546587137 . Added to list
[DEBUG] Found info for tweet:  382816788473729024 . Added to list
[DEBUG] Found info for tweet:  382816798799704064 . Added to list
Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
[DEBUG] Found info for tweet:  382816829875712000 . Added to list
[DEBUG] Found info for tweet:  382816842097516545 . Added to list
[DEBUG] Found info for tweet:  382816843850719232 . Added to list
Twitter API returned a 403 (Forbidden), User has been suspended.
[DEBUG] Found info for tweet:  382816850884960256

[DEBUG] Found info for tweet:  382818547132801024 . Added to list
[DEBUG] Found info for tweet:  382818557643730944 . Added to list
[DEBUG] Found info for tweet:  382818560420360192 . Added to list
[DEBUG] Found info for tweet:  382818570058489857 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
Twitter API returned a 403 (Forbidden), Sorry, you are not authorized to see this status.
[DEBUG] Found info for tweet:  382818604695044097 . Added to list
Twitter API returned a 403 (Forbidden), User has been suspended.
[DEBUG] Found info for tweet:  382818613763112960 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
[DEBUG] Found info for tweet:  382818652929544192 . Added to list
[DEBUG] Found info for tweet:  382818659909246976 . Added to list
[DEBUG] Found info for tweet:  382818672969920513 . Added to list
[DEBUG] Found info for tweet:  382818703777099777 . Added to list
[DEBUG] Found info for tweet:  38281870796757

[DEBUG] Found info for tweet:  382820573673320448 . Added to list
[DEBUG] Found info for tweet:  382820590014693377 . Added to list
[DEBUG] Found info for tweet:  382820595391418368 . Added to list
[DEBUG] Found info for tweet:  382820720708833280 . Added to list
Twitter API returned a 404 (Not Found), No status found with that ID.
Twitter API returned a 404 (Not Found), No status found with that ID.
Twitter API returned a 403 (Forbidden), User has been suspended.
[DEBUG] Found info for tweet:  382820878464983040 . Added to list
[DEBUG] Found info for tweet:  382820883040960512 . Added to list
[DEBUG] Found info for tweet:  382820885280747520 . Added to list
[DEBUG] Found info for tweet:  382820886296145920 . Added to list
[DEBUG] Found info for tweet:  382820887529271296 . Added to list
[DEBUG] Found info for tweet:  382820893656764416 . Added to list
[DEBUG] Found info for tweet:  382820917702713344 . Added to list
[DEBUG] Found info for tweet:  382820921540476928 . Added to list
[DE

...

Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), 

Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), 

Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), 

Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), 

Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), Rate limit exceeded
Twitter API returned a 429 (Too Many Requests), 

In [5]:
list_infos

[{'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
 {'ID': '382827662109986816', 'Followers': None},
